In [ ]:
import pandas as pd
import openpyxl
from pandas import ExcelWriter

In [ ]:
%store -r tarjetas
%store -r bancos

In [ ]:
tarjeta_visa_credito = tarjetas.loc [tarjetas['Cod Marca Producto']== 'Visa'] #Filtro por la Tarjeta Visa Credito
tarjeta_visa_credito = tarjeta_visa_credito.loc[:, (tarjeta_visa_credito!= 0).any(axis=0)]
tarjeta_visa_credito["PERCEPCION IVA R.G. 2408  1,50/3,00 %"] = tarjeta_visa_credito[
                    ['PERCEPCION IVA R.G. 2408   1,50 %', 'PERCEPCION IVA R.G. 2408   3,00 %']].sum(axis=1, min_count=1)
                    #sumo 2 columnas, las de percepcion de IVA
tarjeta_visa_credito['Ventas'] = tarjeta_visa_credito[['VENTAS C/DESCUENTO CONTADO',
                                'VENTAS C/DTO CUOTAS FINANC. OTORG.']].sum(axis=1, min_count=1)
                                #sumo 2 columnas, las de ventas
tarjeta_visa_credito['TOTAL DEDUCCIONES'] = tarjeta_visa_credito[['TOTAL DEDUCCIONES', 'CARGOS DE ENTIDAD']].sum(axis=1, min_count=1)
                                            #sumo 2 columnas, deducciones mas cargos de entidad
#tarjeta_visa_credito['TOTAL DEDUCCIONES'] = tarjeta_visa_credito['TOTAL DEDUCCIONES'] - tarjeta_visa_credito['REEMBOLSOS DE ENTIDAD']

tarjeta_visa_credito['RETENCION ING.BRUTOS SIRTAC'] = (tarjeta_visa_credito['RETENCION ING.BRUTOS SIRTAC']
                                                    + tarjeta_visa_credito['AJUSTE SIRTAC OCT. 1RA QUIN'])
tarjeta_visa_credito['IVA 21%'] = tarjeta_visa_credito[['IVA CRED.FISC.COMERCIO S/ARANC 21,00%', 
                                                        'IVA RI CRED.FISC.COMERCIO S/DTO F.OTORG']].sum(axis=1, min_count=1)
tarjeta_visa_credito['DESCUENTO FINANC.OTORG. CUOTAS/PROMO CUOTAS AHORA 12/18'] = tarjeta_visa_credito[
                    ['DESCUENTO FINANC.OTORG. CUOTAS','PROMO CUOTAS AHORA 12/18']].sum(axis=1, min_count=1)
tarjeta_visa_credito['IVA 10.5%'] = tarjeta_visa_credito[['IVA PROMO CUOTAS AHORA 12/18 - 10,50%',
                    'IVA CRED.FISC.COM.L.25063 S/DTO F.OTOR 10,50']].sum(axis=1, min_count=1)


tarjeta_visa_credito = tarjeta_visa_credito.reindex(columns=['Fecha de Pago', 'Cod Marca Producto', 'Cod Comercio', 'Nro Liquidacion' ,'Ventas', 
'ARANCEL', 'IVA 21%','DESCUENTO FINANC.OTORG. CUOTAS/PROMO CUOTAS AHORA 12/18', 'IVA 10.5%', 'PER B.A.I.BR.DN.01/04',
'PERC.I.B. CORRIENTES R.165/00', 'PERCEP ING BR MISIONES R.G. 003/93','PERCEPCION IVA R.G. 2408  1,50/3,00 %', 'RETENCION IMP.GANANCIAS',
'RETENCION ING.BRUTOS', 'RETENCION IVA','RETENCION ING.BRUTOS SIRTAC', 'TOTAL DEDUCCIONES' ,'TOTAL LIQUIDACION',])
#reordeno columnas, aca ya quedan en el orden que es necesario
tarjeta_visa_credito.insert(7, 'Iva 21 control', 
                          tarjeta_visa_credito['IVA 21%']/tarjeta_visa_credito['ARANCEL'])
                          #agrego columna de control para IVA 21%
tarjeta_visa_credito.insert(10, 'Iva 10.5 control', 
                          tarjeta_visa_credito['IVA 10.5%']/tarjeta_visa_credito['DESCUENTO FINANC.OTORG. CUOTAS/PROMO CUOTAS AHORA 12/18'])
                          #agrego columna de control para IVA 10.5%
tarjeta_visa_credito['ARANCEL'] = tarjeta_visa_credito['IVA 21%']*100/21
tarjeta_visa_credito['DESCUENTO FINANC.OTORG. CUOTAS/PROMO CUOTAS AHORA 12/18'] = tarjeta_visa_credito['IVA 10.5%']*100/10.5
tarjeta_visa_credito = tarjeta_visa_credito.assign(Diferencia = tarjeta_visa_credito.iloc[:, 5:7].sum(axis=True) 
                      + tarjeta_visa_credito.iloc[:, 8:10].sum(axis=True)
                      + tarjeta_visa_credito.iloc[:, 11:19].sum(axis=True)
                      - tarjeta_visa_credito['TOTAL DEDUCCIONES']) #agrego columna de control final


In [ ]:
tarjeta_visa_credito.groupby(by=['Cod Marca Producto', 'Cod Comercio']).agg(group)

In [ ]:
sum_columns = ['Nro Liquidacion', 'Ventas', 'ARANCEL', 'IVA 21%', 'Iva 21 control',
       'DESCUENTO FINANC.OTORG. CUOTAS/PROMO CUOTAS AHORA 12/18', 'IVA 10.5%',
       'Iva 10.5 control', 'PER B.A.I.BR.DN.01/04',
       'PERC.I.B. CORRIENTES R.165/00', 'PERCEP ING BR MISIONES R.G. 003/93',
       'PERCEPCION IVA R.G. 2408  1,50/3,00 %', 'RETENCION IMP.GANANCIAS',
       'RETENCION ING.BRUTOS', 'RETENCION IVA', 'RETENCION ING.BRUTOS SIRTAC',
       'TOTAL DEDUCCIONES', 'TOTAL LIQUIDACION', 'Diferencia']
max_columns = ['Nro Liquidacion']

group = {col: ('max' if col in max_columns else 'sum') for col in sum_columns}

tarjeta_visa_credito=tarjeta_visa_credito.groupby(by=['Cod Marca Producto', 'Cod Comercio']).agg(group)

In [ ]:
tarjeta_visa_credito = tarjeta_visa_credito.drop(['Diferencia', 'Iva 21 control', 'Iva 10.5 control'], axis=1)

In [ ]:
tarjeta_visa_credito = tarjeta_visa_credito.reset_index()
tarjeta_visa_credito = tarjeta_visa_credito.merge(establecimientos)

In [ ]:
writer = ExcelWriter('/content/drive/MyDrive/Colab Notebooks/Le Utthe/Tarjetas/10-22/Visadecredito1022.xlsx')
tarjeta_visa_credito.to_excel(writer, 'Hoja de datos', index=True)
writer.save()